In [99]:
import networkx as nx
import numpy as np
from heapq import heappush, heappop
import time
from memory_profiler import memory_usage
import tsplib95
import csv

### TWICE AROUND THE TREE

In [100]:
def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    caminho = list(nx.dfs_preorder_nodes(MST, 1))
    caminho.append(caminho[0])
    peso = sum(A[caminho[i]][caminho[i + 1]]['weight'] for i in range(len(caminho) - 1))

    return peso, caminho


### CHRISTOFIDES

In [101]:
def encontrarCaminhoCurto(A):
    caminho = [x[0] for x in nx.eulerian_circuit(A, 1)]
    return list(dict.fromkeys(caminho)) + [caminho[0]]

def christofidesTSP(A):
    MST = nx.minimum_spanning_tree(A)
    nosImpares = [no for no, grau in nx.degree(MST) if grau % 2 == 1]
    pareamento = nx.min_weight_matching(nx.subgraph(A, nosImpares))

    MSTMultiGrafo = nx.MultiGraph(MST)
    MSTMultiGrafo.add_edges_from((no1, no2, A[no1][no2]) for no1, no2 in pareamento)

    caminho = encontrarCaminhoCurto(MSTMultiGrafo)
    peso = sum(A[caminho[i]][caminho[i + 1]]['weight'] for i in range(len(caminho) - 1))

    return peso, caminho

#### BRANCH AND BOUND

In [102]:
import numpy as np

class No:
    def __init__(self, limite, arestasLimite, custo, solucao):
        self.limite = limite
        self.arestasLimite = arestasLimite
        self.custo = custo
        self.solucao = solucao
    
    def __lt__(self, outro):
        if len(self.solucao) == len(outro.solucao):
            return self.limite < outro.limite
        return len(self.solucao) > len(outro.solucao)

def encontrarDuasArestasMinimas(lista):
    min1, min2 = np.inf, np.inf
    for j in lista:
        peso = lista[j]['weight']
        if peso < min1:
            min1, min2 = peso, min1
        elif peso < min2:
            min2 = peso
    return min1, min2

def encontrarLimiteInicial(A):
    limite = 0
    arestasLimiteIniciais = np.zeros((A.number_of_nodes(), 2), dtype=object)  # Usando object para armazenar listas
    for i in range(1, A.number_of_nodes() + 1):  # Começando de 1
        min1, min2 = encontrarDuasArestasMinimas(A[i])
        arestasLimiteIniciais[i - 1] = [min1, min2]  # Ajuste no índice para armazenamento
        limite += min1 + min2

    return limite / 2, arestasLimiteIniciais

def encontrarLimite(A, solucao, arestasLimite, limite):
    arestasAlteradas = np.zeros(A.number_of_nodes(), dtype=int)
    novasArestas = np.array(arestasLimite)
    pesoAresta = A[solucao[-2]][solucao[-1]]['weight']
    soma = limite * 2

    for no in solucao[-2:]:
        no_index = no - 1  # Ajuste para a indexação começando em 1
        if novasArestas[no_index][0] != pesoAresta:
            soma -= novasArestas[no_index][arestasAlteradas[no_index]]
            soma += pesoAresta
            arestasAlteradas[no_index] += 1

    return soma / 2, novasArestas


In [103]:
def branchAndBoundTSP(A):
    limiteInicial, arestasLimiteIniciais = encontrarLimiteInicial(A)
    raiz = No(limiteInicial, arestasLimiteIniciais, 0, [1])  # Iniciando com [1]
    heap = []
    heappush(heap, raiz)
    melhorCusto = np.inf
    melhorSolucao = []
    contadorDeNos = 0

    while heap:
        noAtual = heappop(heap)
        contadorDeNos += 1
        nivel = len(noAtual.solucao)

        if nivel > A.number_of_nodes():
            if melhorCusto > noAtual.custo:
                melhorCusto = noAtual.custo
                melhorSolucao = noAtual.solucao
        else:
            if noAtual.limite < melhorCusto:
                if nivel < A.number_of_nodes() - 2:
                    for k in range(1, A.number_of_nodes() + 1):
                        if k in noAtual.solucao:
                            continue
                        pesoAresta = A[noAtual.solucao[-1]][k]['weight']
                        novoLimite, novasArestas = encontrarLimite(A, noAtual.solucao + [k], noAtual.arestasLimite, noAtual.limite)
                        if novoLimite < melhorCusto:
                            novoNo = No(novoLimite, novasArestas, noAtual.custo + pesoAresta, noAtual.solucao + [k])
                            heappush(heap, novoNo)
                else:
                    for k in range(1, A.number_of_nodes() + 1):
                        if k in noAtual.solucao:
                            continue
                        ultimoNo = next(i for i in range(1, A.number_of_nodes() + 1) if i not in noAtual.solucao + [k] and k != i)
                        pesoAresta = A[noAtual.solucao[-1]][k]['weight']
                        proxPesoAresta = A[k][ultimoNo]['weight']
                        ultimoPesoAresta = A[ultimoNo][1]['weight']  # Ajuste para o primeiro nó sendo 1
                        custo = noAtual.custo + pesoAresta + proxPesoAresta + ultimoPesoAresta
                        if custo < melhorCusto:
                            novoNo = No(custo, [], custo, noAtual.solucao + [k, ultimoNo, 1])  # Ajuste para o primeiro nó sendo 1
                            heappush(heap, novoNo)

    return melhorCusto, melhorSolucao


#### Criando grafos de teste

In [104]:
A = np.array([[0, 3, 1, 5, 8],
    [3, 0, 6, 7, 9],
    [1, 6, 0, 4, 2],
    [5, 7, 4, 0, 3],
    [8, 9, 2, 3, 0]])

A = nx.from_numpy_array(np.matrix(A), create_using=nx.Graph)

B = np.array([[0, 4, 8, 9, 12],
    [4, 0, 6, 8, 9],
    [8, 6, 0, 10, 11],
    [9, 8, 10, 0, 7],   
    [12, 9, 11, 7, 0]])

B = nx.from_numpy_array(np.matrix(B), create_using=nx.Graph)

C = nx.complete_graph(15)
for (u, v, w) in C.edges(data=True):
    w['weight'] = np.random.randint(1, 10)

In [105]:
def executarAlgoritmo(algoritmo, dados, peso_otimo):
    tempo_inicio = time.time()
    tempo_maximo = 30 * 60  # 30 minutos em segundos
    resultado_dict = {
        'algoritmo': algoritmo.__name__,
        'peso': None,
        'caminho': None,
        'tempo_execucao': None,
        'uso_memoria': None,
        'desvio_percentual': None
    }

    try:
        uso_memoria = memory_usage((algoritmo, (dados,)))
        resultado = algoritmo(dados)
        tempo_fim = time.time()

        tempo_execucao = tempo_fim - tempo_inicio
        if tempo_execucao > tempo_maximo:
            raise TimeoutError

        peso_algoritmo = resultado[0]
        desvio_percentual = ((peso_algoritmo - peso_otimo) / peso_otimo) * 100

        resultado_dict.update({
            'peso': peso_algoritmo,
            'caminho': resultado[1],
            'tempo_execucao': tempo_execucao,
            'uso_memoria': np.max(uso_memoria),
            'desvio_percentual': desvio_percentual
        })

        return resultado_dict
    except TimeoutError:
        resultado_dict['peso'] = "NA"
        return resultado_dict


#executarAlgoritmo(twiceAroundTheTreeTSP, A)
#executarAlgoritmo(christofidesTSP, B)
#executarAlgoritmo(branchAndBoundTSP, C)


In [106]:
def carregar_problema_tsp(caminho_arquivo):
    problema = tsplib95.load(caminho_arquivo)
    grafo = problema.get_graph()
    return grafo

def carregar_solucao_optima(caminho_arquivo):
    with open(caminho_arquivo, 'r') as file:
        linhas = file.readlines()
        tour = [int(linha.strip()) for linha in linhas if linha.strip().isdigit()]
    return tour

def calcular_peso_solucao_optima(grafo, tour_optima):
    peso_total = 0
    for i in range(len(tour_optima) - 1):
        peso_total += grafo[tour_optima[i]][tour_optima[i + 1]]['weight']
    # Adiciona o peso para retornar ao ponto de partida
    peso_total += grafo[tour_optima[-1]][tour_optima[0]]['weight']
    return peso_total

def salvar_em_csv(dados, nome_arquivo, nome_dataset):
    with open(nome_arquivo, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Escreve o cabeçalho se o arquivo estiver vazio
        if file.tell() == 0:
            writer.writerow(['Dataset', 'Algoritmo', 'Peso', 'Caminho', 'Tempo de Execução', 'Uso de Memória', 'Desvio Percentual'])

        writer.writerow([nome_dataset, dados['algoritmo'], dados['peso'], dados['caminho'], 
                         dados['tempo_execucao'], dados['uso_memoria'], dados['desvio_percentual']])


nome_dataset = 'berlin52'
nome_arquivo_csv = 'resultados_tsp.csv'

# Supondo que as funções carregar_problema_tsp, carregar_solucao_optima e calcular_peso_solucao_optima estejam definidas
grafo = carregar_problema_tsp(f'datasets/{nome_dataset}.tsp')
tour_otima = carregar_solucao_optima(f'datasets/{nome_dataset}.opt.tour')
peso_otimo = calcular_peso_solucao_optima(grafo, tour_otima)

# Executar algoritmos e salvar resultados
for algoritmo in [twiceAroundTheTreeTSP, christofidesTSP, branchAndBoundTSP]:
    resultado = executarAlgoritmo(algoritmo, grafo, peso_otimo)
    salvar_em_csv(resultado, nome_arquivo_csv, nome_dataset)


KeyboardInterrupt: 